In [1]:
from data_utils import MergedMelHarmDataset, GenCollator
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
from torch.utils.data import DataLoader
from transformers import RobertaConfig, RobertaForMaskedLM
import torch
from torch.optim import AdamW
from tqdm import tqdm

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies/'

In [3]:
chordSymbolTokenizer = ChordSymbolTokenizer.from_pretrained('saved_tokenizers/ChordSymbolTokenizer')
rootTypeTokenizer = RootTypeTokenizer.from_pretrained('saved_tokenizers/RootTypeTokenizer')
pitchClassTokenizer = PitchClassTokenizer.from_pretrained('saved_tokenizers/PitchClassTokenizer')
rootPCTokenizer = RootPCTokenizer.from_pretrained('saved_tokenizers/RootPCTokenizer')
gctRootPCTokenizer = GCTRootPCTokenizer.from_pretrained('saved_tokenizers/GCTRootPCTokenizer')
gctSymbolTokenizer = GCTSymbolTokenizer.from_pretrained('saved_tokenizers/GCTSymbolTokenizer')
gctRootTypeTokenizer = GCTRootTypeTokenizer.from_pretrained('saved_tokenizers/GCTRootTypeTokenizer')
melodyPitchTokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')

In [4]:
m_chordSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, chordSymbolTokenizer)
m_rootTypeTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootTypeTokenizer)
m_pitchClassTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, pitchClassTokenizer)
m_rootPCTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootPCTokenizer)
m_gctRootPCTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctRootPCTokenizer)
m_gctSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctSymbolTokenizer)
m_gctRootTypeTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctRootTypeTokenizer)

In [5]:
tokenizer = m_chordSymbolTokenizer

max_length=2048

dataset = MergedMelHarmDataset(root_dir, tokenizer, max_length=max_length, return_harmonization_labels=True)
collator = GenCollator(tokenizer)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collator)


In [6]:
b = next(iter(dataloader))
print(b)

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


{'input_ids': tensor([[  2,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1],
        ...,
        [  2,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]])}


In [7]:
b['input_ids'].shape

torch.Size([16, 2048])

In [8]:
x = b['labels'][0][ b['labels'][0] != -100 ]
print( x )

tensor([  6,  95, 233,   6,  95, 233,   6,  95, 233,   6,  95, 233,   6,  95,
        214,   6,  95, 214,   6,  95, 214,   6,  95, 214,   6,  95, 233,   6,
         95, 233,   6,  95, 233,   6,  95, 233,   6,  95, 436,   6,  95, 436,
          6,  95, 436,   6,  95, 436,   6,  95, 378,   6,  95, 378,   6,  95,
        378,   6,  95, 378,   6,  95, 350,   6,  95, 378,   6,  95, 350,   6,
         95, 378,   6,  95, 413,   6,  95, 204,   3])


In [9]:
tokenizer.convert_ids_to_tokens(b['input_ids'][0].tolist())

['<s>',
 '<bar>',
 'ts_4x4',
 'position_0x00',
 'P:67',
 'position_2x00',
 'P:67',
 'position_2x50',
 'P:72',
 '<bar>',
 'position_0x00',
 'P:77',
 'position_1x50',
 'P:76',
 'position_2x00',
 'P:76',
 'position_3x50',
 'P:75',
 '<bar>',
 'position_0x00',
 'P:75',
 'position_1x50',
 'P:72',
 'position_2x00',
 'P:72',
 '<bar>',
 'position_0x00',
 'P:72',
 'position_2x00',
 'P:71',
 'position_2x50',
 'P:66',
 'position_3x00',
 'P:61',
 'position_3x50',
 'P:60',
 '<bar>',
 'position_0x00',
 'P:60',
 'position_2x00',
 'P:60',
 'position_2x50',
 'P:65',
 '<bar>',
 'position_0x00',
 'P:70',
 'position_1x50',
 'P:74',
 'position_2x00',
 'P:74',
 'position_3x00',
 'P:70',
 '<bar>',
 'position_0x00',
 'P:70',
 'position_0x50',
 'P:65',
 'position_2x00',
 'P:70',
 '<bar>',
 'position_0x00',
 'P:70',
 '<bar>',
 'position_0x00',
 '<rest>',
 '<bar>',
 'position_0x00',
 '<rest>',
 'position_1x00',
 '<rest>',
 'position_2x00',
 '<rest>',
 '<bar>',
 'position_1x00',
 '<rest>',
 'position_2x00',
 '<res

In [10]:
tokenizer.convert_ids_to_tokens(x.tolist())

['<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C:min11',
 '<bar>',
 'position_0x00',
 'C:min11',
 '<bar>',
 'position_0x00',
 'C:min11',
 '<bar>',
 'position_0x00',
 'C:min11',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'C#:maj7',
 '<bar>',
 'position_0x00',
 'G#:maj7',
 '<bar>',
 'position_0x00',
 'G#:maj7',
 '<bar>',
 'position_0x00',
 'G#:maj7',
 '<bar>',
 'position_0x00',
 'G#:maj7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'F:min7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'F:min7',
 '<bar>',
 'position_0x00',
 'F#:maj7',
 '<bar>',
 'position_0x00',
 'G:hdim7',
 '

In [11]:
print(tokenizer.melody_tokenizer.vocab)

{'<unk>': 0, '<pad>': 1, '<s>': 2, '</s>': 3, '<rest>': 4, '<mask>': 5, '<bar>': 6, 'P:21': 7, 'P:22': 8, 'P:23': 9, 'P:24': 10, 'P:25': 11, 'P:26': 12, 'P:27': 13, 'P:28': 14, 'P:29': 15, 'P:30': 16, 'P:31': 17, 'P:32': 18, 'P:33': 19, 'P:34': 20, 'P:35': 21, 'P:36': 22, 'P:37': 23, 'P:38': 24, 'P:39': 25, 'P:40': 26, 'P:41': 27, 'P:42': 28, 'P:43': 29, 'P:44': 30, 'P:45': 31, 'P:46': 32, 'P:47': 33, 'P:48': 34, 'P:49': 35, 'P:50': 36, 'P:51': 37, 'P:52': 38, 'P:53': 39, 'P:54': 40, 'P:55': 41, 'P:56': 42, 'P:57': 43, 'P:58': 44, 'P:59': 45, 'P:60': 46, 'P:61': 47, 'P:62': 48, 'P:63': 49, 'P:64': 50, 'P:65': 51, 'P:66': 52, 'P:67': 53, 'P:68': 54, 'P:69': 55, 'P:70': 56, 'P:71': 57, 'P:72': 58, 'P:73': 59, 'P:74': 60, 'P:75': 61, 'P:76': 62, 'P:77': 63, 'P:78': 64, 'P:79': 65, 'P:80': 66, 'P:81': 67, 'P:82': 68, 'P:83': 69, 'P:84': 70, 'P:85': 71, 'P:86': 72, 'P:87': 73, 'P:88': 74, 'P:89': 75, 'P:90': 76, 'P:91': 77, 'P:92': 78, 'P:93': 79, 'P:94': 80, 'P:95': 81, 'P:96': 82, 'P:97':

In [12]:
print(tokenizer.harmony_tokenizer.vocab)

{'<unk>': 0, '<pad>': 1, '<s>': 2, '</s>': 3, '<emp>': 195, '<mask>': 5, '<bar>': 6, '<h>': 196, 'position_0x00': 95, 'position_0x16': 96, 'position_0x25': 97, 'position_0x33': 98, 'position_0x50': 99, 'position_0x66': 100, 'position_0x75': 101, 'position_0x83': 102, 'position_1x00': 103, 'position_1x16': 104, 'position_1x25': 105, 'position_1x33': 106, 'position_1x50': 107, 'position_1x66': 108, 'position_1x75': 109, 'position_1x83': 110, 'position_2x00': 111, 'position_2x16': 112, 'position_2x25': 113, 'position_2x33': 114, 'position_2x50': 115, 'position_2x66': 116, 'position_2x75': 117, 'position_2x83': 118, 'position_3x00': 119, 'position_3x16': 120, 'position_3x25': 121, 'position_3x33': 122, 'position_3x50': 123, 'position_3x66': 124, 'position_3x75': 125, 'position_3x83': 126, 'position_4x00': 127, 'position_4x16': 128, 'position_4x25': 129, 'position_4x33': 130, 'position_4x50': 131, 'position_4x66': 132, 'position_4x75': 133, 'position_4x83': 134, 'position_5x00': 135, 'posit

In [13]:
import torch
from transformers import AutoConfig, GPT2LMHeadModel, AdamW

# define model
vocab_size = tokenizer.total_vocab_size
d_model = 256
num_heads = 4
num_layers = 4
d_ff = 256
max_seq_length = max_length
dropout = 0.3

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=vocab_size,
    n_positions=max_seq_length,
    n_layer=num_layers,
    n_head=num_heads,
    pad_token_id=tokenizer.vocab[tokenizer.pad_token],
    bos_token_id=tokenizer.vocab[tokenizer.bos_token],
    eos_token_id=tokenizer.vocab[tokenizer.eos_token],
    n_embd=d_ff
)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# Load model
model = GPT2LMHeadModel(config).to(device)
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    total_loss = 0
    num_batches = 0

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        num_batches += 1

        total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / num_batches}")

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")


/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Epoch 1/3, Loss: 6.329387664794922
Epoch 1/3, Loss: 5.99749493598938
Epoch 1/3, Loss: 5.751000881195068
Epoch 1/3, Loss: 5.555291295051575
Epoch 1/3, Loss: 5.413634490966797
Epoch 1/3, Loss: 5.2875228722890215
Epoch 1/3, Loss: 5.180427483149937
Epoch 1/3, Loss: 5.098048388957977
Epoch 1/3, Loss: 5.014269298977322
Epoch 1/3, Loss: 4.937037467956543
Epoch 1/3, Loss: 4.8680971752513535
Epoch 1/3, Loss: 4.800151705741882
Epoch 1/3, Loss: 4.742569153125469
Epoch 1/3, Loss: 4.700528383255005
Epoch 1/3, Loss: 4.654326947530111
Epoch 1/3, Loss: 4.617589861154556
Epoch 1/3, Loss: 4.572910477133358
Epoch 1/3, Loss: 4.529044601652357
Epoch 1/3, Loss: 4.491002936112253


KeyboardInterrupt: 